In [141]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

In [142]:
claravine_folder_path = r'..\..\Dell-Projects\Helper\Claravine.csv'
calendar_folder_path = r'..\..\Dell-Projects\Helper\Calendar.csv'
delve_folder_path = r'C:\Users\Rafael_Fagundes\Downloads\compiled_data.csv'

claravine_df = pd.read_csv(claravine_folder_path, parse_dates=['created_date'])
delve_df = pd.read_csv(delve_folder_path)
calendar_df = pd.read_csv(calendar_folder_path, parse_dates=['Date'])

In [143]:
filtered_claravine_df = claravine_df[['Segment ID', 'MediaCom Audience Name', 'Country', 'Business Unit', 'Audience Source', 'created_date']]
filtered_delve_df = delve_df[['Segment ID', 'Media Buy Name']][(delve_df['Segment ID'] != '0') & (delve_df['Segment ID'] != 0)]

grouped_delve_df = filtered_delve_df.groupby(['Segment ID'])['Media Buy Name'].count().reset_index()
grouped_delve_df = grouped_delve_df.rename(columns={'Media Buy Name': 'Placements Count'})

# Use 'merge' to combine the DataFrames based on 'Segment ID'
merged = filtered_claravine_df.merge(grouped_delve_df, on='Segment ID', how='left')

# Fill NaN values in 'Placements Count' with 0
merged['Placements Count'].fillna(0, inplace=True)

# Create 'Segment Used' column based on the presence of 'Segment ID' in 'merged'
merged['Segment Used'] = np.where(merged['Segment ID'].isin(grouped_delve_df['Segment ID']), 1, 0)

merged.head()


,Segment ID,MediaCom Audience Name,Country,Business Unit,Audience Source,created_date,Placements Count,Segment Used
0,14o04dsyat,NaN,us,csbo-aw,Google Analytics,2023-01-17 18:56:46,0.0,0
1,da2z3jfdx3d,NaN,us,csbo-csb,Google Analytics,2023-01-19 14:45:00,0.0,0
2,d2l072rynmf,targeting-development_us-csbo-csb_1pcrm-aiqcrm...,us,csbo-csb,AIQ CRM,2023-01-20 17:49:12,0.0,0
3,dztqn3bkuwh,targeting-gamingpcbuyer_us-csbo-csb_1pcrm-aiqc...,us,csbo-csb,AIQ CRM,2023-01-20 17:49:12,0.0,0
4,dfmb6iq56n5,targeting-mbinmarket_us-mb_abm-liveramp-dunand...,us,mb,Liveramp,2023-01-23 20:09:10,0.0,0


In [144]:
# Define the mapping of Business Units to BU_lvl1 values
bu_mapping = {
    'b2b': 'B2B',
    'b2b-com': 'B2B',
    'b2b-mb': 'B2B',
    'ce': 'B2B',
    'com': 'B2B',
    'mb': 'B2B',
    'csbo-aw': 'CSB',
    'csbo-aw/gseries': 'CSB',
    'csbo-con': 'CSB',
    'csbo-csb': 'CSB',
    'csbo-gaming': 'CSB',
    'csbo-gseries': 'CSB',
    'csbo-sb': 'CSB',
    'sb': 'CSB',
    'brand':'PAN-DELL',
    'pan-dell':'PAN-DELL',
}

# Use apply to map Business Unit to BU_lvl1
merged['BU lvl1'] = merged['Business Unit'].apply(lambda x: bu_mapping.get(x.lower(), x))


In [145]:
calendar_df.rename(columns={'Date': 'created_date'}, inplace=True)

merged['created_date'] = pd.to_datetime(merged['created_date']).dt.strftime("%Y-%m-%d")
merged['created_date'] = pd.to_datetime(merged['created_date'])

In [146]:
filtered_calendar_df = calendar_df[['created_date','FQ','FQW']]

In [147]:
new_merged = pd.merge(merged, filtered_calendar_df, on='created_date', how='left')

new_merged.rename(columns={'FQW':'Created Date Fiscal Week','created_date':'Created Date','Business Unit':'BU lvl2'}, inplace=True)

In [148]:
new_merged['Segment Used'].value_counts()

Segment Used
0    741
1    183
Name: count, dtype: int64

In [149]:
new_merged.to_csv('csv/Segments IDs Used.csv')